# **Atividade de Análise de Algoritmos**

#  Comparar Djikstra vs A* 

Aluna: Flávia Gaia.

Professor: Rodrigo Hagstrom.

Data de entrega: 4 de jun. 10:00 

# **Djikstra**

https://www.youtube.com/watch?v=fdqxetN9YPA

In [ ]:
import math
class HeapMin:

    def __init__(self):
        self.nos = 0
        self.heap = []

    def adiciona_no(self, u, indice):
        self.heap.append([u, indice])
        self.nos += 1
        f = self.nos
        while True:
            if f == 1:
                break
            p = f // 2
            if self.heap[p-1][0] <= self.heap[f-1][0]:
                break
            else:
                self.heap[p-1], self.heap[f-1] = self.heap[f-1], self.heap[p-1]
                f = p

    def mostra_heap(self):
        print('A estrutura heap é a seguinte:')
        nivel = int(math.log(self.nos, 2))
        a = 0
        for i in range(nivel):
            for j in range(2 ** i):
                print(f'{self.heap[a]}', end='  ')
                a += 1
            print('')
        for i in range(self.nos-a):
            print(f'{self.heap[a]}', end='  ')
            a += 1
        print('')

    def remove_no(self):
        x = self.heap[0]
        self.heap[0] = self.heap[self.nos - 1]
        self.heap.pop()
        self.nos -= 1
        p = 1
        while True:
            f = 2 * p
            if f > self.nos:
                break
            if f + 1 <= self.nos:
                if self.heap[f][0] < self.heap[f-1][0]:
                    f += 1
            if self.heap[p-1][0] <= self.heap[f-1][0]:
                break
            else:
                self.heap[p-1], self.heap[f-1] = self.heap[f-1], self.heap[p-1]
                p = f
        return x

    def tamanho(self):
        return self.nos

    def menor_elemento(self):
        if self.nos != 0:
            return self.heap[0]
        return 'A árvore está vazia'

    def filho_esquerda(self, u):
        if self.nos >= 2*u:
            return self.heap[2*u-1]
        return 'Esse nó não tem filho'

    def filho_direita(self, u):
        if self.nos >= 2*u+1:
            return self.heap[2*u]
        return 'Esse nó não tem filho da direita'

    def pai(self, u):
        return self.heap[u // 2]


class Grafo:

    def __init__(self, vertices):
        self.vertices = vertices
        self.grafo = [[0] * self.vertices for i in range(self.vertices)]

    def adiciona_aresta(self, u, v, peso):
        self.grafo[u-1][v-1] = peso
        self.grafo[v-1][u-1] = peso

    def mostra_matriz(self):
        print('A matriz de adjacências é:')
        for i in range(self.vertices):
            print(self.grafo[i])

    def dijkstra(self, origem):
        custo_vem = [[-1, 0] for i in range(self.vertices)]
        custo_vem[origem - 1] = [0, origem]
        h = HeapMin()
        h.adiciona_no(0, origem)
        while h.tamanho() > 0:
            dist, v = h.remove_no()
            for i in range(self.vertices):
                if self.grafo[v-1][i] != 0:
                    if custo_vem[i][0] == -1 or custo_vem[i][0] > dist + self.grafo[v-1][i]:
                        custo_vem[i] = [dist + self.grafo[v-1][i], v]
                        h.adiciona_no(dist + self.grafo[v-1][i], i+1)
        return custo_vem





g = Grafo(7)

g.adiciona_aresta(1, 2, 5)
g.adiciona_aresta(1, 3, 6)
g.adiciona_aresta(1, 4, 10)
g.adiciona_aresta(2, 5, 13)
g.adiciona_aresta(3, 4, 3)
g.adiciona_aresta(3, 5, 11)
g.adiciona_aresta(3, 6, 6)
g.adiciona_aresta(4, 5, 6)
g.adiciona_aresta(4, 6, 4)
g.adiciona_aresta(5, 7, 3)
g.adiciona_aresta(6, 7, 8)

g.mostra_matriz()

resultado_dijkstra = g.dijkstra(1)
print(resultado_dijkstra)

A matriz de adjacências é:
[0, 5, 6, 10, 0, 0, 0]
[5, 0, 0, 0, 13, 0, 0]
[6, 0, 0, 3, 11, 6, 0]
[10, 0, 3, 0, 6, 4, 0]
[0, 13, 11, 6, 0, 0, 3]
[0, 0, 6, 4, 0, 0, 8]
[0, 0, 0, 0, 3, 8, 0]
[[0, 1], [5, 1], [6, 1], [9, 3], [15, 4], [12, 3], [18, 5]]


# **A estrela**

https://gist.github.com/nenodias/d92b4cdbfb92ace257ff535856ba0a46

In [ ]:
#Matriz visual usada pra montar a matriz de adjacência
matriz = [ [0, 1, 2],
           [3, 4, 5],
           [6, 7, 8], ]
'''
  0  |  1     2
             --- 
  3     4     5
 ---  
  6     7  |  8
Ir do 0 ao 8
sem bater nas paredes
'''

# Variável onde seu índice representa a posíção no labirinto
# E seu valor representa os possíveis lugares para se mover
nos = [
    [3],#nó 0
    [4, 2],#nó 1
    [1],#nó 2
    [4, 0],#nó 3
    [7, 5, 1],#nó 4
    [8, 4, 2],#nó 5
    [7],#nó 6
    [6, 4],#nó 7
    [5] #nó 8
]

inicio = 0
final = 8


#Função para testar qual a distância espacial do nó n para o nó final
def retorna_h(destino, node):
    def h(n):
        def caminhar(pos, visitados, distancia):
            caminhos = node[pos]
            distancias = []
            visitados.add(pos)
            if pos == destino:
                return distancia
            distancia += 1
            for novo_pos in caminhos:
                if novo_pos == destino:
                    return distancia
                if not novo_pos in visitados:
                    try:
                        distancias.append( caminhar(novo_pos,set(visitados), distancia) )
                    except:
                        pass
            return min(distancias)


        try:
            return caminhar(n,set(), 0)
        except:
            return None
                    
    return h

meu_h = retorna_h(final, nos)

print( meu_h(0) )

def retorna_caminho(destino, node, meu_h):
    def meu_f(n):
        def calculo_f(pos, g_anterior):
            valor = meu_h(pos)
            if not valor:
                raise Exception('Não há caminho para determinado destino')
            return g_anterior + valor
        pos = n
        g = 0
        path = [pos]
        try:
            while True:
                menor = 999999
                idx_menor = -1
                caminhos = node[pos]
                for caminho in caminhos:
                    if caminho == destino:
                        path.append(caminho)
                        return path
                    if caminho in path:
                        continue
                    heuristica = calculo_f(caminho, g+1)
                    if heuristica < menor:
                        menor = heuristica
                        idx_menor = caminho
                if idx_menor != -1:
                    pos = idx_menor
                    path.append(idx_menor)
                    g += 1
                    continue
                else:
                    break
        except:
            return []
        return path
    return meu_f

calcular_caminho = retorna_caminho(final, nos, meu_h)


texto = ''
caminhos = calcular_caminho(0)
for i in range( len(caminhos) ):
    passagem = caminhos[i]
    if i != 0:
        texto += ' => '
    texto += str(passagem)

print(texto)

4
0 => 3 => 4 => 5 => 8
